Вам дан объект класса SomeObject, содержащего три поля: integer_field, float_field и string_field:

In [1]:
class SomeObject:
    def __init__(self):
        self.integer_field = 0
        self.float_field = 0.0
        self.string_field = ""

Необходимо реализовать поведение:

EventGet(<type>) создаёт событие получения данных соответствующего типа
EventSet(<value>) создаёт событие изменения поля типа type(<value>)
Необходимо реализовать классы NullHandler, IntHandler, FloatHandler, StrHandler так, чтобы можно было создать цепочку:
    
    chain = IntHandler(FloatHandler(StrHandler(NullHandler)))

Описание работы цепочки:  

chain.handle(obj, EventGet(int)) — вернуть значение obj.integer_field

chain.handle(obj, EventGet(str)) — вернуть значение obj.string_field

chain.handle(obj, EventGet(float)) — вернуть значение obj.float_field

chain.handle(obj, EventSet(1)) — установить значение obj.integer_field =1

chain.handle(obj, EventSet(1.1)) — установить значение obj.float_field = 1.1

chain.handle(obj, EventSet("str")) — установить значение obj.string_field = "str"


In [2]:
INT, FLOAT, STR = int, float, str

In [3]:
class EventGet:
    def __init__(self, value):
        self.kind = {int: INT, float: FLOAT, str: STR}[value]
        self.value = None

In [4]:
class EventSet:
    def __init__(self, value):
        self.kind = {int: INT, float: FLOAT, str: STR}[type(value)]
        self.value = value

In [5]:
class NullHandler:
    def __init__(self, successor=None):
        self.__successor = successor

    def handle(self, some_obj, event):
        """Сам обработчик"""
        if self.__successor is not None:  # Если следующий обработчик вообще существует, передадим ему это событие
            return self.__successor.handle(some_obj, event)

In [6]:
class IntHandler(NullHandler):
    def handle(self, some_obj, event):
        if event.kind == INT:
            if event.value is None:
                return some_obj.integer_field
            else:
                some_obj.integer_field = event.value
        else:
            return super().handle(some_obj, event)

In [7]:
class FloatHandler(NullHandler):
    def handle(self, some_obj, event):
        if event.kind == FLOAT:
            if event.value is None:
                return some_obj.float_field
            else:
                some_obj.float_field = event.value
        else:
            return super().handle(some_obj, event)

In [8]:
class StrHandler(NullHandler):
    def handle(self, some_obj, event):
        if event.kind == STR:
            if event.value is None:
                return some_obj.string_field
            else:
                some_obj.string_field = event.value
        else:
            return super().handle(some_obj, event)

In [9]:
if __name__ == "__main__":
    obj = SomeObject()

    obj.integer_field = 42
    obj.float_field = 3.14
    obj.string_field = "some text"

    chain = IntHandler(FloatHandler(StrHandler(NullHandler())))

    print(chain.handle(obj, EventGet(int)))
    print(chain.handle(obj, EventGet(float)))
    print(chain.handle(obj, EventGet(str)))

    chain.handle(obj, EventSet(100))
    chain.handle(obj, EventSet(0.5))
    chain.handle(obj, EventSet('new text'))

    print(chain.handle(obj, EventGet(int)))
    print(chain.handle(obj, EventGet(float)))
    print(chain.handle(obj, EventGet(str)))

42
3.14
some text
100
0.5
new text
